<a href="https://colab.research.google.com/github/caroacostatovany/sentiment_analysis_tweets/blob/main/notebooks/limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTEBOOK DE LIMPIEZA

In [26]:
!pip install "git+https://github.com/ElenaVillano/sentiment_analysis_tweets.git#egg=nlptweet&subdirectory=src" --quiet

In [25]:
!pip uninstall nlptweet

Uninstalling nlptweet-0.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/nlp/*
    /usr/local/lib/python3.7/dist-packages/nlptweet-0.1.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled nlptweet-0.1


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
import string
# Nuestro paquete
import nlp


#call the nltk downloader
nltk.download('punkt')

from dateutil import parser

#from sklearn.model_selection import train_test_split

# Carga un set de stopwords predefinidas
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# Nombramiento de columnas
col_names = ['target', # Polaridad del twet 0=negativo, 2=neutral, 4=positivo
             'ids',    # ID tweet
             'date',   # Fecha y hora del tweet
             'flag',   # QUERY
             'user',   # Usuario del tweet
             'text']   # Texto del tweety

# Carga y limpieza de datos

In [4]:
# Requiered to select a file to be imported into colab
# Not useful if running locally
from google.colab import files
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv


In [94]:
data =  pd.read_csv('training.1600000.processed.noemoticon.csv',
                 encoding='latin-1', names=col_names)

In [95]:
print(data.shape)

(1600000, 6)


In [96]:
# Ejemplo
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
27,0,1467814883,Mon Apr 06 22:20:52 PDT 2009,NO_QUERY,gagoo,im sad now Miss.Lilly
41,0,1467818481,Mon Apr 06 22:21:46 PDT 2009,NO_QUERY,lionslamb,He's the reason for the teardrops on my guitar...
44,0,1467819022,Mon Apr 06 22:21:56 PDT 2009,NO_QUERY,hpfangirl94,Falling asleep. Just heard about that Tracy gi...
35,0,1467816749,Mon Apr 06 22:21:20 PDT 2009,NO_QUERY,scarletletterm,ok I'm sick and spent an hour sitting in the s...
48,0,1467820206,Mon Apr 06 22:22:13 PDT 2009,NO_QUERY,peacoats,is strangely sad about LiLo and SamRo breaking...
155,0,1467845157,Mon Apr 06 22:28:51 PDT 2009,NO_QUERY,styletrain,Man Work is Hard


In [97]:
data['raw_text'] = data['text']

## Convertimos a minúsculas

In [98]:
from nlp.preprocessing import convierte_a_minusculas

In [99]:
data = convierte_a_minusculas(data)

In [100]:
# Ejemplo
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,@tatiana_k nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now miss.lilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep. just heard about that tracy gi...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Quitamos \&quot; 

In [101]:
from nlp.preprocessing import quitar_caracteres_html

In [102]:
data['text'] = data['text'].map(lambda s: quitar_caracteres_html(s))

## Reemplazamos las URLs con el texto "URL"

In [103]:
from nlp.preprocessing import reemplazar_urls

In [106]:
data['text'] = data['text'].map(lambda s: reemplazar_urls(s))

In [105]:
# Ejemplo
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,@tatiana_k nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now miss.lilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep. just heard about that tracy gi...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Reemplazamos las menciones @ con la palabra "USER_MENTION"



In [107]:
from nlp.preprocessing import reemplazar_usuarios

In [109]:
data['text'] = data['text'].map(lambda s: reemplazar_usuarios(s))

In [110]:
# Ejemplo
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,"no, it's not behaving at all. i'm mad. why am...","@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now miss.lilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep. just heard about that tracy gi...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Quitamos el Hashtag , pero dejamos la palabra

In [111]:
# The regular expression used to match hashtags is #(\S+).
from nlp.preprocessing import quitar_hashtag

In [112]:
data['text'] = data['text'].map(lambda s: quitar_hashtag(s))

In [113]:
# Ejemplo
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,"no, it's not behaving at all. i'm mad. why am...","@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now miss.lilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep. just heard about that tracy gi...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Quitamos los Retweets, sólo la palabra RT , dejamos el comentario

In [114]:
#  The regular expression used to match retweets is \brt\b.
from nlp.preprocessing import quitar_RT

In [115]:
data['text'] = data['text'].map(lambda s: quitar_RT(s))

In [116]:
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,"no, it's not behaving at all. i'm mad. why am...","@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now miss.lilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep. just heard about that tracy gi...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Quitamos espacios o puntos extras

In [117]:
## Reemplazamos los dobles puntos (o más) con un espacio , dos o más espacios con 1 espacio y hacemos strip de espacios y comillas
# Strip any punctuation [’"?!,.():;-'] from the word ??
from nlp.preprocessing import quitar_caracteres_especiales

In [118]:
data['text'] = data['text'].map(lambda s: quitar_caracteres_especiales(s))

In [119]:
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,no it's not behaving at all i'm mad why am i ...,"@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now misslilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep just heard about that tracy gir...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Quitamos expresiones con letras repetidas

In [120]:
#Convert 2 or more letter repetitions to 2 letters.
#Some people send tweets like I am sooooo
#happpppy adding multiple characters to emphasize
#on certain words. This is done to handle such tweets
#by converting them to I am soo happy
from nlp.preprocessing import quitar_letras_repetidas

In [121]:
data['text'] = data['text'].map(lambda s: quitar_letras_repetidas(s))

In [122]:
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,no it's not behaving at all i'm mad why am i ...,"@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now misslilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep just heard about that tracy gir...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Quitar caracteres nonascii

In [123]:
from nlp.preprocessing import quitar_nonascii

In [124]:
data['text'] = data['text'].map(lambda s: quitar_nonascii(s))

In [125]:
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,no it's not behaving at all i'm mad why am i ...,"@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they didn't have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now misslilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he's the reason for the teardrops on my guitar...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep just heard about that tracy gir...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i'm sick and spent an hour sitting in the s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


## Removemos stopwords

In [126]:
from nlp.preprocessing import separar_abreviaciones

In [127]:
data['text'] = data['text'].map(lambda s: separar_abreviaciones(s))

In [128]:
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,no it is not behaving at all i am mad why am ...,"@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope they did not have it,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad now misslilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,he is the reason for the teardrops on my guita...,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep just heard about that tracy gir...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok i am sick and spent an hour sitting in the ...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,is strangely sad about lilo and samro breaking...,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work is hard,man work is hard


In [129]:
from nlp.preprocessing import remove_stopwords

In [130]:
data['text'] = data['text'].map(lambda s: remove_stopwords(s))

In [131]:
data.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text,raw_text
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,behaving mad see,"@nationwideclass no, it's not behaving at all...."
8,0,1467811795,mon apr 06 22:20:05 pdt 2009,no_query,2hood4hollywood,nope,@tatiana_k nope they didn't have it
27,0,1467814883,mon apr 06 22:20:52 pdt 2009,no_query,gagoo,im sad misslilly,im sad now miss.lilly
41,0,1467818481,mon apr 06 22:21:46 pdt 2009,no_query,lionslamb,reason teardrops guitar one enough break heart,he's the reason for the teardrops on my guitar...
44,0,1467819022,mon apr 06 22:21:56 pdt 2009,no_query,hpfangirl94,falling asleep heard tracy girl body found sad...,falling asleep. just heard about that tracy gi...
35,0,1467816749,mon apr 06 22:21:20 pdt 2009,no_query,scarletletterm,ok sick spent hour sitting shower cause sick s...,ok i'm sick and spent an hour sitting in the s...
48,0,1467820206,mon apr 06 22:22:13 pdt 2009,no_query,peacoats,strangely sad lilo samro breaking,is strangely sad about lilo and samro breaking...
155,0,1467845157,mon apr 06 22:28:51 pdt 2009,no_query,styletrain,man work hard,man work is hard


## Stemming con NLTK

In [132]:
from nlp.preprocessing import oracion_raiz

In [ ]:
data['text'] = data['text'].map(lambda s: oracion_raiz(s))

In [ ]:
data

In [83]:
from nlp.preprocessing import modificar_etiqueta

In [ ]:
data = modificar_etiqueta(data)

In [ ]:
data

In [86]:
data.to_csv("1600000_datos_limpios_sin_user_ni_url.csv")

### Escogemos 100,000 k del mismo largo de palabras

In [87]:
data['tokens'] = data['text'].map(lambda s: s.split())

In [ ]:
data

In [89]:
data['len_tokens'] = data['tokens'].map(lambda s: len(s))

In [ ]:
data.sort_values('len_tokens', ascending=False)